# Copurchase Network of Amazon Books

In [1]:
import pickle
import numpy as np
from igraph import *
import sqlite3 as lite

### Create edge list of books

#### Connect to database

In [3]:
amazon_db = lite.connect("../AmazonBooks/amazon.db")
amazon_db.text_factory = str
cur=amazon_db.cursor()
cur.execute("select book1, book2 from cop_list")
items = cur.fetchall()   

#### Convert ISBN to ASIN

ASIN is the identifier for books used by Amazon. Different editions of a book have different ISBNs, but comform to the same one ASIN.

In [12]:
isbn2asin = pickle.load(open('../AmazonBooks/isbn_lookup.pkl'))
edgeList=map(lambda x: (isbn2asin[x[0]], isbn2asin[x[1]]), items)

### Create copurchase network of books

In [28]:
asin2id=dict(zip(list(set(isbn2asin.values())),range(len(set(isbn2asin.values())))))
g = Graph()
g.add_vertices(list(set(isbn2asin.values())))
g.add_edges(edgeList)
g.simplify()

In [56]:
cur.execute("select isbn, title, rank from unique_book_attribute")
book_entries = cur.fetchall()
for book in book_entries:
    g.vs[asin2id[isbn2asin[book[0]]]]['title']=book[1]

In [58]:
with open('../AmazonBooks/asin_network.pkl','wb') as outfile:
    pickle.dump(g,outfile)
pickle.dump(asin2id,open('../AmazonBooks/asin2id.pkl','wb'))

In [3]:
amazon_db = lite.connect("../AmazonBooks/amazon.db")
amazon_db.text_factory = str
cur=amazon_db.cursor()
cur.execute("select isbn, title, rank from unique_book_attribute")
book_entries = cur.fetchall()
isbns,titles,ranks=zip(*book_entries)

In [4]:
bookTitles=dict(zip(isbns,titles))
bookRanks=dict(zip(isbns,ranks))

In [5]:
liberals=set()
with open('../BookCodes/liberals.txt') as l:
    for line in l:
        liberals.add(line.strip())

conservatives=set()
with open('../BookCodes/conservatives.txt') as c:
    for line in c:
        conservatives.add(line.strip())

In [22]:
titleRanks=dict()
for i in conservatives:
    titleRanks[bookTitles[i]]=bookRanks[i]
    
df=pd.DataFrame(sorted(titleRanks.items(),key=lambda x:x[1]))
df.to_csv('../AmazonBooks/conservativeBooks.tsv', sep='\t', index=False)

In [13]:
import pandas as pd